# Exp4: 基于K-近邻的车牌号识别

## 一、案例简介

图像的智能处理一直是人工智能领域广受关注的一类技术，代表性的如人脸识别与 CT 肿瘤识别，在人工智能落地的进程中发挥着重要作用。其中车牌号识别作为一个早期应用场景，已经融入日常生活中，为我们提供了诸多便利，在各地的停车场和出入口都能看到它的身影。车牌号识别往往分为字符划分和字符识别两个子任务，本案例我们将关注字符识别的任务，尝试用 K-NN 的方法对分割好的字符图像进行自动识别和转化。

## 二、作业说明

### 基本要求
* 完成数据的读入和表示，将图片表示成向量并和 label 对应上；
* 构建 K-NN 模型（可调库）对测试集中的图片进行预测并计算准确率；
* 分析当 K 取不同值时测试准确率的变化。

### 扩展要求
* 分析不同距离度量方式对模型效果的影响；
* 对比平权和加权 K-NN 的效果；
* 分析训练集大小对测试结果的影响。

## 三、数据概览
本次我们使用已经分割好的车牌图片作为数据集，包括数字 0-9、字母 A-Z（不包含 O 和 I）以及省份简称共 65 个类，编号从 0 到 64。数据已经分成了训练集和测试集，里面的文件夹用 label 编号命名，一个文件夹下的所有图片都属于该文件夹对应的类，每个图片都是 20 * 20 的二值化灰度图。

下面演示一下如何借助 PIL 库将图片转化为向量：

In [3]:
from PIL import Image
img = Image.open('data/train/0/4-3.jpg')  # 打开图片
img  # 显示图片from PIL import Image
img = Image.open('data/train/0/4-3.jpg')  # 打开图片
img  # 显示图片

In [4]:
import numpy as np
pixels = np.array(img)  # 转化为 numpy 矩阵
pixels.shape

(20, 20)

## 四、模型构建

In [3]:
import os
import numpy as np
from PIL import Image

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# 完成数据的读入和表示，将图片表示成向量并和 label 对应上
def load_dataset(root_dir):
    X = []
    y = []

    for label_name in sorted(os.listdir(root_dir)):
        label_path = os.path.join(root_dir, label_name)
        if not os.path.isdir(label_path):
            continue

        label = int(label_name)

        for file in os.listdir(label_path):
            img_path = os.path.join(label_path, file)
            img = Image.open(img_path).convert('L')  # 灰度
            pixels = np.array(img)
            pixels.shape  # (height, width)

            # 二值图已经是 0/255 或 0/1，这里统一拉平
            X.append(pixels.flatten())
            y.append(label)

    return np.array(X), np.array(y)

X_train, y_train = load_dataset('data/train')
X_test, y_test = load_dataset('data/test')

def evaluate_knn(k):
    knn = KNeighborsClassifier(
        n_neighbors=k,
        weights='distance',   # 距离加权
        metric='euclidean'    # 欧氏距离
    )
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    return acc

K_values = range(1, 21)
accuracies = []

for k in K_values:
    acc = evaluate_knn(k)
    accuracies.append(acc)
    print(f"K={k}, Test Accuracy={acc:.4f}")

K=1, Test Accuracy=0.7168
K=2, Test Accuracy=0.7168
K=3, Test Accuracy=0.7104
K=4, Test Accuracy=0.7078
K=5, Test Accuracy=0.7016
K=6, Test Accuracy=0.6992
K=7, Test Accuracy=0.6992
K=8, Test Accuracy=0.6958
K=9, Test Accuracy=0.6950
K=10, Test Accuracy=0.6943
K=11, Test Accuracy=0.6930
K=12, Test Accuracy=0.6932
K=13, Test Accuracy=0.6881
K=14, Test Accuracy=0.6845
K=15, Test Accuracy=0.6821
K=16, Test Accuracy=0.6806
K=17, Test Accuracy=0.6770
K=18, Test Accuracy=0.6761
K=19, Test Accuracy=0.6718
K=20, Test Accuracy=0.6707


**分析当 K 取不同值时测试准确率的变化：**
- K=1（1-3）：方差大，对噪声、笔画断裂敏感
- K 适中（5-9）：平衡偏差–方差，类内结构被稳定捕捉
- K 太大（>15）：不同字符类（如 B/8、0/D、省份简称）被“平均”掉，会导致偏差上升

### 控制变量调整训练集大小

In [5]:

from sklearn.model_selection import StratifiedShuffleSplit

# Find the best K from the first evaluation
best_k = K_values[np.argmax(accuracies)]
print(f"\nBest K value: {best_k} with accuracy: {accuracies[np.argmax(accuracies)]:.4f}\n")

def evaluate_with_ratio(ratio, k, repeats=5):
    knn = KNeighborsClassifier(
        n_neighbors=k,
        weights='distance',   # 距离加权
        metric='euclidean'    # 欧氏距离
    )
    accs = []
    
    # If ratio is 1.0, use full training set directly
    if ratio >= 1.0:
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        return acc, 0.0  # No std for single evaluation
    
    for _ in range(repeats):
        sss = StratifiedShuffleSplit(
            n_splits=1, train_size=ratio, random_state=None
        )
        for train_idx, _ in sss.split(X_train, y_train):
            X_sub = X_train[train_idx]
            y_sub = y_train[train_idx]

            knn.fit(X_sub, y_sub)
            y_pred = knn.predict(X_test)
            accs.append(accuracy_score(y_test, y_pred))
    return np.mean(accs), np.std(accs)

# Evaluate with different training set ratios
ratios = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
ratio_results = []

for ratio in ratios:
    mean_acc, std_acc = evaluate_with_ratio(ratio, best_k)
    ratio_results.append((ratio, mean_acc, std_acc))
    print(f"Ratio={ratio:.1f}, Test Accuracy={mean_acc:.4f} ± {std_acc:.4f}")



Best K value: 1 with accuracy: 0.7168

Ratio=0.1, Test Accuracy=0.6373 ± 0.0264
Ratio=0.2, Test Accuracy=0.6520 ± 0.0123
Ratio=0.3, Test Accuracy=0.6878 ± 0.0146
Ratio=0.4, Test Accuracy=0.6893 ± 0.0072
Ratio=0.5, Test Accuracy=0.6949 ± 0.0099
Ratio=0.6, Test Accuracy=0.7067 ± 0.0109
Ratio=0.7, Test Accuracy=0.7144 ± 0.0145
Ratio=0.8, Test Accuracy=0.7134 ± 0.0046
Ratio=0.9, Test Accuracy=0.7140 ± 0.0059
Ratio=1.0, Test Accuracy=0.7168 ± 0.0000


**可见，随着训练集规模的增加，kNN（K=1）在字符识别任务上的测试准确率整体呈单调上升趋势，并在使用约 70% 以上训练数据后趋于饱和，最终稳定在约 71.7%，表明模型性能主要受限于数据分布本身及字符间的不可约相似性。**